<a href="https://colab.research.google.com/github/amolk/AGI-experiments/blob/master/Pattern%20Machine/02_Pattern_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pattern machine
- continuous, i.e. activation changes over time as decaying history of instantaneous activation
- modular, i.e. connect up more flexibly,
- use multi-patterns, i.e. pattern contains more than 1 weights tensor. This is to represent input and output jointly.

Architectural decisions
- Signal, SignalGrid, CompositeSignalGrid
- input patches may overlap when utility factor > 1, fixed number (a grid) of patterns per patch
- output neighborhood is the patterns corresponding to a neighborhood of patches

In [ ]:
!pip install ipytest

     |████████████████████████████████| 276kB 6.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4


In [ ]:
%load_ext autoreload
%autoreload 1
import ipytest
ipytest.autoconfig()

In [ ]:
%%writefile utils.py
import pdb
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d

def pretty_s(name, clas, indent=0):
  if type(clas).__name__ == 'Tensor':
    return ' ' * indent + name + ":" + type(clas).__name__ + " size" + str(tuple(clas.shape))

  strs = []
  strs.append(' ' * indent + name + ":" + type(clas).__name__)

  indent += 2
  for k,v in clas.__dict__.items():
    if '__dict__' in dir(v):
      strs.append(pretty_s(k, v,indent))
    elif '__iter__' in dir(v):
      if type(v) is tuple:
        strs.append(' ' * indent + k + ' = ' + str(v))
      else:
        strs.append(' ' * indent +  k + ' = [')
        for index, item in enumerate(v):
          if '__dict__' in dir(item):
            strs.append(pretty_s(str(index), item, indent+2))
          else:
            strs.append(' ' * (indent+2) + str(item))
        strs.append(' ' * indent +  ']')
    else:
      strs.append(' ' * indent +  k + ' = ' + str(v))

  return "\n".join(strs)
          
def pretty_print(name, clas, indent=0):
  print(pretty_s(name, clas, indent))

def soft_add(a, b, tau):
  return a * (1 - tau) + b * tau

def add_gaussian_noise(tensor, mean=0., std=1.):
    t = tensor + torch.randn(tensor.size()).to(device) * std + mean
    t.to(device)
    return t

def plot_patterns(patterns, pattern_lr, dataset, voronoi=False, annotate=False, figsize=(7,7), dpi=100):
  patterns = patterns.cpu()
  dataset = dataset.cpu()
  assert len(patterns.shape) == 2 # (pattern count, 2)
  assert patterns.shape[1] == 2 # 2D

  rgba_colors = torch.zeros((patterns.shape[0], 4))

  # for blue the last column needs to be one
  rgba_colors[:,2] = 1.0
  # the fourth column needs to be your alphas
  if pattern_lr is not None:
    alpha = (1.1 - pattern_lr.cpu()).clamp(0, 1) * 0.9
    rgba_colors[:, 3] = alpha
  else:
    rgba_colors[:, 3] = 1.0

  plt.figure(figsize=figsize, dpi=dpi)
  ax = plt.gca()
  ax.cla() # clear things for fresh plot

  if annotate:
    for i in range(patterns.shape[0]):
      ax.annotate(str(i), (patterns[i][0], patterns[i][1]), xytext=(5,-3), textcoords='offset points')

  ax.scatter(patterns[:, 0], patterns[:, 1], marker='.', c=rgba_colors, s=50)
  ax.scatter(dataset[:, 0], dataset[:, 1], marker='.', c='r', s=10)

  if voronoi:
    vor = Voronoi(patterns)
    vor_fig = voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='gray',
                              line_width=1, line_alpha=0.2, point_size=0)

  ax.set_xlim(0, 1)
  ax.set_ylim(0, 1)
  plt.show()

"""
Create a numpy array of given shape, each element initialized using supplied function
Example: make_ndarray((2,3), lambda multi_index:multi_index)
"""
def make_ndarray(shape, fn):
  a = np.empty(shape, dtype=object)
  with np.nditer(a, flags=['refs_ok', 'multi_index'], op_flags=['readwrite']) as it:
    for x in it:
      a[it.multi_index] = fn(it.multi_index)

  return a

Overwriting utils.py


In [ ]:
%aimport utils

In [ ]:
%%writefile pattern.py

import torch
import numpy
import pdb
from typing import List
from utils import pretty_s

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class GridShapeMismatchError(Exception): pass
class NoComponentsError(Exception): pass

class SignalUtils:
  @staticmethod
  def compute_precision(variance):
    return torch.exp(-variance)

class SignalGridHP:
  def __init__(self, grid_shape, signal_shape, init_pixel_scale=0.1, init_variance=10):
    self.grid_shape = grid_shape
    self.grid_size = numpy.prod(self.grid_shape)
    if self.grid_size <= 0:
      raise ValueError("Invalid grid size")

    self.signal_shape = signal_shape
    self.signal_size = numpy.prod(self.signal_shape)
    if self.signal_size <= 0:
      raise ValueError("Invalid signal size")

    self.init_pixel_scale = init_pixel_scale
    self.init_variance = init_variance

class SignalGrid:
  def __init__(self, hp:SignalGridHP, alloc_pixels=True, pixels=None):
    self.hp = hp
    if pixels is not None:
      self.pixels = pixels
    elif alloc_pixels:
      self.pixels = torch.rand((hp.grid_size, hp.signal_size)).to(device) * hp.init_pixel_scale
    else:
      self.pixels = None

    self.variance = torch.ones((hp.grid_size, hp.signal_size)).to(device) * hp.init_variance
    self.precision = SignalUtils.compute_precision(self.variance)

  @property
  def signal_shape(self):
    return self.hp.signal_shape

  @property
  def __dict__(self):
    return {
        'grid_shape': self.hp.grid_shape,
        'signal_shape': self.hp.signal_shape,
        'pixels': self.pixels,
        'precision': self.precision
    }

  def __repr__(self):
    return pretty_s("", self)

class CompositeSignalGridHP:
  def __init__(self, hps:List[SignalGridHP]):
    if len(hps) == 0:
      raise NoComponentsError("Must specify at least one component")

    self.components = hps
    self.grid_shape = hps[0].grid_shape

    # all components must have same grid size
    for component_hp in hps:
      if component_hp.grid_shape != hps[0].grid_shape:
        raise GridShapeMismatchError

class CompositeSignalGrid:
  @staticmethod
  def from_tensors(signals:List[torch.Tensor], variance:float=0.0):
    signal_hps = [SignalGridHP(grid_shape=(1,), signal_shape=signal.shape, init_variance=variance) for signal in signals]
    hp = CompositeSignalGridHP(hps=signal_hps)
    result = CompositeSignalGrid(hp=hp, alloc=False)
    for index, component in enumerate(result.components):
      component.pixels = signals[index].unsqueeze(0)
    return result

  def from_signal_grids(signal_grids:List[SignalGrid]):
    result = CompositeSignalGrid(hp=None)
    hp = CompositeSignalGridHP(hps=[sg.hp for sg in signal_grids])
    result.hp = hp
    result.components = signal_grids
    return result

  def __init__(self, hp:CompositeSignalGridHP=None, alloc=True):
    self.hp = hp
    if hp:
      self.grid_shape = hp.grid_shape
      self.components = [SignalGrid(component_hp, alloc_pixels=alloc) for component_hp in hp.components]

  # add a SignalGrid as a component  
  def add_component(self, o:SignalGrid):
    assert self.hp.grid_shape == o.hp.grid_shape

    self.hp.components.append(o.hp)
    self.components.append(o)

  @property
  def signal_shape(self):
    return [c.hp.signal_shape for c in self.components]

  @property
  def component_count(self):
    return len(self.components)

  def __repr__(self):
    return pretty_s("", self)

class PatternGridHP:
  def __init__(self, grid_shape, composite_signal_grid_hp:CompositeSignalGridHP):
    self.grid_shape = grid_shape
    self.grid_size = numpy.prod(self.grid_shape)

    self.composite_signal_grid_hp = composite_signal_grid_hp
    for component in composite_signal_grid_hp.components:
      if component.grid_shape != self.grid_shape:
        raise GridShapeMismatchError

class PatternGrid:
  def __init__(self, hp:PatternGridHP):
    self.hp = hp
    self.composite_signal_grid = CompositeSignalGrid(hp.composite_signal_grid_hp)
    self.alpha = torch.ones((hp.grid_size,)).to(device)

  def __repr__(self):
    return pretty_s("", self)

class PatternSimilarityHP:
  def __init__(self, enable_precision_weighted_distance=True):
    self.enable_precision_weighted_distance = enable_precision_weighted_distance

class PatternSimilarity:
  def __init__(self, x:CompositeSignalGrid, y:CompositeSignalGrid, hp:PatternSimilarityHP=None):
    """
    Find similarity between each item in x with each item in y.
    grid_sizes (i.e. number of items) for x and y may differ.
    signal shapes of x and y must match.
    """
    if hp:
      self.hp = hp
    else:
      self.hp = PatternSimilarityHP()

    assert x.component_count == y.component_count, f"x.component_count {x.component_count} != y.component_count {y.component_count}"
    assert x.signal_shape == y.signal_shape, f"x.signal_shape {x.signal_shape} != y.signal_shape {y.signal_shape}"

    self.dist_1 = []
    self.dist_d = []
    self.dist = []
    self.sim_components = []

    # find similarity based on each signal component
    for component_index in range(x.component_count):
      x_component = x.components[component_index]
      y_component = y.components[component_index]
      dist_1_component, dist_d_component, dist_component = self.l2_distance(
          x=x_component.pixels,
          x_precision=x_component.precision,
          y=y_component.pixels,
          y_precision=y_component.precision)
      sim_component = torch.exp(-dist_component)

      self.dist_1.append(dist_1_component)
      self.dist_d.append(dist_d_component)
      self.dist.append(dist_component)
      self.sim_components.append(sim_component)

    # final similarity is mean of signal component similarities
    # this equalizes class weights for all components (e.g. modalities)
    self.sim = torch.stack(self.sim_components).mean(dim=0)

  def l2_distance(self, x, x_precision, y, y_precision):
    xs = x.shape
    assert len(xs) == 2
    assert (x_precision is None) or (x_precision.shape == xs), "Precision, if specified, must be same shape as patterns"

    ys = y.shape
    assert len(ys) == 2
    assert (y_precision is None) or (y_precision.shape == ys), "Precision, if specified, must be same shape as patterns"

    assert xs[1] == ys[1], "Patch size, i.e. dim 1, must match"

    n = xs[0]
    m = ys[0]
    d = xs[1]

    x = x.unsqueeze(1).expand(n, m, d)
    y = y.unsqueeze(0).expand(n, m, d)

    dist_1 = (x - y).abs()
    dist_d = torch.pow(dist_1, 2)

    if self.hp.enable_precision_weighted_distance:
      if x_precision is not None:
        dist_d = dist_d * x_precision.unsqueeze(1).expand(n, m, d)

      if y_precision is not None:
        dist_d = dist_d * y_precision.unsqueeze(0).expand(n, m, d)

    dist = dist_d.sum(2).sqrt()
    return dist_1, dist_d, dist


Overwriting pattern.py


In [ ]:
%aimport pattern

In [ ]:
%%writefile convoluation_utils.py
import math
from pattern import *
from typing import List, Tuple

class ConvolutionUtils:
  @staticmethod
  def make_afferent_patches(signal:CompositeSignalGrid, grid_shape:Tuple, coverage_factor:float=1.0):
    # print("make_afferent_patches")
    # print("  signal", signal.signal_shape)
    # print("  grid_shape", grid_shape)
    # print("  coverage_factor", coverage_factor)

    assert signal.grid_shape == (1,)
    for grid_shape_i in grid_shape:
      assert grid_shape_i > 1

    patch_signal_grids = []

    for component_index in range(len(signal.components)):
      # print("  component", component_index)
      component = signal.components[component_index]

      patch_shape = tuple([int(coverage_factor *  component.signal_shape[i] / grid_shape[i]) for i in range(len(grid_shape))])
      # print("    patch_shape", patch_shape)
      stride = tuple([math.floor((component.signal_shape[i]-patch_shape[i])/(grid_shape[i]-1)) for i in range(len(grid_shape))])
      # print("    stride", stride)

      patches = ConvolutionUtil.conv_slice(component.pixels.view((1,) + component.hp.signal_shape), patch_shape, stride=stride).squeeze(dim=0)
      sghp = SignalGridHP(grid_shape=grid_shape, signal_shape=patch_shape)
      patch_signal_grid = SignalGrid(hp=sghp, alloc_pixels=False, pixels = patches)
               
      # print("  patch_signal_grid", patch_signal_grid)
      patch_signal_grids.append(patch_signal_grid)

    patches = CompositeSignalGrid.from_signal_grids(patch_signal_grids)
    return patches

  def make_neighborhood_patches(signal:SignalGrid, patch_shape:Tuple):
    # print("make_neighborhood_patches")
    # print("  signal", signal)
    # print("  patch_shape", patch_shape)

    assert signal.hp.grid_shape == (1,)

    stride = 1 #(1,) * len(signal.signal_shape)
    # print("  stride", stride)

    padding = tuple([int((i-1)/2) for i in patch_shape])
    # print("  padding", padding)

    patches = ConvolutionUtil.conv_slice(signal.pixels.view((1,) + signal.hp.signal_shape), patch_shape, stride=stride, padding=padding).squeeze(dim=0)
    sghp = SignalGridHP(grid_shape=signal.signal_shape, signal_shape=patch_shape)
    patch_signal_grid = SignalGrid(hp=sghp, alloc_pixels=False, pixels = patches)
              
    # print("  patch_signal_grid", patch_signal_grid)

    return patch_signal_grid


  @staticmethod
  def conv_slice(images, kernel_shape, stride, padding=0):
    assert len(images.shape) == 3, "Must be (image count, image height, image width)"
    images = images.unsqueeze(1)

    fold_params = dict(kernel_size=kernel_shape, dilation=1, padding=padding, stride=stride)
    unfold = torch.nn.Unfold(**fold_params)
    # print("images", images.shape)
    unfolded = unfold(images)
    unfolded = unfolded.view(images.shape[0], -1, unfolded.shape[-1])
    unfolded = unfolded.transpose(1, 2)
    return unfolded

Overwriting convoluation_utils.py


In [ ]:
%aimport convoluation_utils

In [ ]:
%%run_pytest[clean]
"""
Test SignalGrid
"""

import pytest
import pdb
from pattern import *

def test_signal_grid_invalid_grid_size1():
  with pytest.raises(ValueError):
    SignalGridHP(
      grid_shape=(0,4),     # <-- zero
      signal_shape=(5,6,2)
    )

def test_signal_grid_invalid_grid_size2():
  with pytest.raises(ValueError):
    SignalGridHP(
      grid_shape=(1,4),
      signal_shape=(5,-1,2) # <-- negative
    )

@pytest.fixture
def signal_grid_hp1():
  return SignalGridHP(
      grid_shape=(3,4),
      signal_shape=(5,6,2))

@pytest.fixture
def signal_grid1(signal_grid_hp1):
  return SignalGrid(hp=signal_grid_hp1)

def test_create_signal_grid(signal_grid_hp1):
  signal_grid = SignalGrid(hp=signal_grid_hp1)
  assert signal_grid.pixels.shape == (3*4, 5*6*2)
  assert signal_grid.variance.shape == signal_grid.pixels.shape
  assert signal_grid.precision.shape == signal_grid.pixels.shape

@pytest.fixture
def signal_grid_hp_degenerate():
  return SignalGridHP(
      grid_shape=(1),
      signal_shape=(1))

def test_create_signal_grid_degenerate(signal_grid_hp_degenerate):
  signal_grid = SignalGrid(hp=signal_grid_hp_degenerate)
  assert signal_grid.pixels.shape == (1, 1)
  assert signal_grid.variance.shape == signal_grid.pixels.shape
  assert signal_grid.precision.shape == signal_grid.pixels.shape

def test_composite_signal_grid_from_tensors():
  csg = CompositeSignalGrid.from_tensors([torch.ones((10,10)), torch.ones((5,5))])
  assert csg.grid_shape == (1,)
  assert len(csg.components) == 2

  c0 = csg.components[0]
  assert c0.hp.grid_shape == (1,)
  assert c0.signal_shape == (10,10)

  c1 = csg.components[1]
  assert c1.hp.grid_shape == (1,)
  assert c1.signal_shape == (5,5)

def test_composite_signal_grid_from_signal_grids_error1():
  grid_shape0 = (3,4)
  signal_shape0 = (5,3,2)
  grid_shape1 = (1,2)
  signal_shape1 = (12)

  sgs = [
         SignalGrid(hp=SignalGridHP(grid_shape=grid_shape0, signal_shape=signal_shape0)), 
         SignalGrid(hp=SignalGridHP(grid_shape=grid_shape1, signal_shape=signal_shape1))
  ]
  with pytest.raises(GridShapeMismatchError):
    CompositeSignalGrid.from_signal_grids(sgs)

def test_composite_signal_grid_from_signal_grids():
  grid_shape = (3,4)
  signal_shape0 = (5,3,2)
  signal_shape1 = (12)

  sgs = [
         SignalGrid(hp=SignalGridHP(grid_shape=grid_shape, signal_shape=signal_shape0)), 
         SignalGrid(hp=SignalGridHP(grid_shape=grid_shape, signal_shape=signal_shape1))
  ]
  csg = CompositeSignalGrid.from_signal_grids(sgs)
  
  assert csg.hp.grid_shape == grid_shape
  assert len(csg.components) == 2

  c0 = csg.components[0]
  assert c0.hp.grid_shape == grid_shape
  assert c0.signal_shape == signal_shape0

  c1 = csg.components[1]
  assert c1.hp.grid_shape == grid_shape
  assert c1.signal_shape == signal_shape1


In [ ]:
%%run_pytest[clean]
"""
Test CompositeSignalGrid
"""

import pytest
from pattern import *

def test_csg_zero_components():
  """
  Must have at least 1 component
  """
  with pytest.raises(NoComponentsError):
    CompositeSignalGridHP([])

def test_csg_different_grid_shapes():
  """
  Components may not have different grid shapes
  """
  with pytest.raises(GridShapeMismatchError):
    CompositeSignalGridHP([
      SignalGridHP(grid_shape=(1,2), signal_shape=(3,4)),
      SignalGridHP(grid_shape=(2,2), signal_shape=(3,4))
    ])

@pytest.fixture
def composite_signal_grid1():
  return CompositeSignalGrid(CompositeSignalGridHP([
    SignalGridHP(grid_shape=(1,2), signal_shape=(3,4)),
    SignalGridHP(grid_shape=(1,2), signal_shape=(3,2,1))
  ]))

def test_csg_different_signal_shapes(composite_signal_grid1):
  """
  Components may have different signal shapes
  """
  assert composite_signal_grid1.component_count == 2
  assert composite_signal_grid1.components[0].signal_shape == (3,4)
  assert composite_signal_grid1.components[1].signal_shape == (3,2,1)
  assert composite_signal_grid1.signal_shape == [(3,4), (3,2,1)]

In [ ]:
%%run_pytest[clean]
"""
Test PatternGrid
"""

import pytest
from pattern import *

@pytest.fixture
def pg1():
  grid_shape = (1,2)
  csg_hp = CompositeSignalGridHP([
    SignalGridHP(grid_shape=grid_shape, signal_shape=(3,4)),
    SignalGridHP(grid_shape=grid_shape, signal_shape=(3,2,1))
  ])

  pg_hp = PatternGridHP(
      grid_shape=grid_shape,
      composite_signal_grid_hp= csg_hp
  )

  return PatternGrid(hp=pg_hp)

def test_pg_create(pg1):
  pg1.alpha.shape == (1,2)

@pytest.fixture
def pg2():
  grid_shape = (1,)
  csg_hp = CompositeSignalGridHP([
    SignalGridHP(grid_shape=grid_shape, signal_shape=(1,)),
    SignalGridHP(grid_shape=grid_shape, signal_shape=(1,))
  ])

  pg_hp = PatternGridHP(
      grid_shape=grid_shape,
      composite_signal_grid_hp= csg_hp
  )

  return PatternGrid(hp=pg_hp)

def test_pg_create_2(pg2):
  pg2.alpha.shape == (1,)


In [ ]:
%%run_pytest[clean] -s
"""
Test Similarity
"""

import pytest
from pattern import *
from utils import *

def make_pg():
  grid_shape = (2,4)
  csg_hp = CompositeSignalGridHP([
    SignalGridHP(grid_shape=grid_shape, signal_shape=(3,4)),
    SignalGridHP(grid_shape=grid_shape, signal_shape=(3,2,1))
  ])

  pg_hp = PatternGridHP(
      grid_shape=grid_shape,
      composite_signal_grid_hp= csg_hp
  )

  return PatternGrid(hp=pg_hp)

def make_pg2():
  grid_shape = (3,2)
  csg_hp = CompositeSignalGridHP([
    SignalGridHP(grid_shape=grid_shape, signal_shape=(3,4)),
    SignalGridHP(grid_shape=grid_shape, signal_shape=(3,2,1))
  ])

  pg_hp = PatternGridHP(
      grid_shape=grid_shape,
      composite_signal_grid_hp= csg_hp
  )

  return PatternGrid(hp=pg_hp)

def test_sim_low_precision():
  # By default, very low precision, so equal even when pixels are random
  pgs = [make_pg(), make_pg2()]
  sim = PatternSimilarity(pgs[0].composite_signal_grid, pgs[1].composite_signal_grid)
  assert sim.sim.shape == (pgs[0].hp.grid_size, pgs[1].hp.grid_size)
  assert torch.allclose(sim.sim, torch.ones_like(sim.sim))

def test_sim_disable_precision_weighting():
  # If disabled precision weighting, then unequal because pixels are random
  pgs = [make_pg(), make_pg2()]
  hp = PatternSimilarityHP(enable_precision_weighted_distance=False)
  sim = PatternSimilarity(pgs[0].composite_signal_grid, pgs[1].composite_signal_grid, hp=hp)
  assert sim.sim.shape == (pgs[0].hp.grid_size, pgs[1].hp.grid_size)
  assert not torch.allclose(sim.sim, torch.ones_like(sim.sim))

def test_sim_high_precision():
  # If high precision, then unequal given pixels are random
  pgs = [make_pg(), make_pg2()]

  # force high precisions
  for pg in pgs:
    for component in pg.composite_signal_grid.components:
      component.precision = torch.ones_like(component.precision)

  sim = PatternSimilarity(pgs[0].composite_signal_grid, pgs[1].composite_signal_grid)
  assert sim.sim.shape == (pgs[0].hp.grid_size, pgs[1].hp.grid_size)
  assert not torch.allclose(sim.sim, torch.ones_like(sim.sim))

In [ ]:
%%writefile layer.py

import torch
import numpy
import pdb
from typing import List, Tuple
from pattern import *
import math
from utils import *
from convoluation_utils import ConvolutionUtils

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class CompositeSignalPatchGrid:
  def __init__(self, signal:CompositeSignalGrid, grid_shape:Tuple, coverage_factor=1.0):
    # signal, i.e. composite image, must be a single composite signal, i.e. its grid shape must be (1,)
    assert signal.grid_shape == (1,)

    self.patches:CompositeSignalGrid = ConvolutionUtils.make_afferent_patches(signal=signal, grid_shape=grid_shape, coverage_factor=coverage_factor)

class NeighborhoodPatchGrid:
  def __init__(self, signal:SignalGrid, patch_shape:Tuple):
    # signal must be a single signal, i.e. its grid shape must be (1,)
    assert signal.grid_shape == (1,)

    # makes one patch per element of the image using neighborhood around the element, i.e. output grid shape = image shape
    self.patches:SignalGrid = ConvolutionUtils.make_neighborhood_patches(signal=signal, patch_shape=patch_shape)

class InputOutputPatchGrid:
  def __init__(self, input:CompositeSignalGrid, output:SignalGrid, output_patch_shape:Tuple, input_coverage_factor=1.0):
    input_grid_shape = output.signal_shape

    self.input_patches = CompositeSignalPatchGrid(signal=input, grid_shape=input_grid_shape, coverage_factor=input_coverage_factor)
    self.output_patches = NeighborhoodPatchGrid(signal=output, patch_shape=output_patch_shape)
    
    self.patches = self.input_patches.patches
    self.patches.add_component(self.output_patches.patches)

# A grid of patterns for each patch in a grid of patches
class PatchGridPatterns:
  def __init__(self, patch_grid_shape:Tuple, per_patch_pattern_grid_hp:PatternGridHP):
    self.patterns = make_ndarray(patch_grid_shape, lambda multi_index: PatternGrid(hp=per_patch_pattern_grid_hp))

class LayerHP:
  def __init__(self, input_signal_shapes:List[Tuple], input_coverage_factor:float, output_shape:Tuple, per_patch_pattern_grid_shape:Tuple, output_neighborhood_shape:Tuple, output_tau=0.5):
    self.input_signal_shapes = input_signal_shapes
    self.input_coverage_factor = input_coverage_factor
    self.output_shape = output_shape
    self.per_patch_pattern_grid_shape = per_patch_pattern_grid_shape
    self.output_neighborhood_shape = output_neighborhood_shape
    self.output_tau = output_tau

    for size in self.output_neighborhood_shape:
      assert size % 2 == 1, "Output neighborhood shape must be odd, so can be centered around specific output activation"

    self.output_hp = SignalGridHP(grid_shape=(1,), signal_shape=output_shape, init_pixel_scale=0.0)

    # sample_input_signal = [torch.ones(shape) for shape in input_signal_shapes]
    # self.convolve_input = convolve_input
    # sample_input = CompositeSignalGrid.from_tensors(sample_input_signal)
    # patches = LayerUtils.make_input_patches(input=sample_input, output_shape=output_shape)

    # # input grid HP
    # hps = [
    #   SignalGridHP(
    #       grid_shape=output_shape, # Each grid element in input grid produces 1 pixel of output
    #       signal_shape=input_component.signal_shape)
    #   for input_component in patches.components # each component of input
    # ]
    # self.input_grid_hp = CompositeSignalGridHP(hps=hps.copy())

    # # pattern HP
    # hps.append(SignalGridHP(
    #     grid_shape=output_shape,
    #     signal_shape=output_neighborhood_shape))

    # self.pattern_grid_hp = PatternGridHP(
    #     grid_shape=pattern_grid_shape,
    #     composite_signal_grid_hp=CompositeSignalGridHP(hps=[
    #       SignalGridHP(grid_shape=pattern_grid_shape, signal_shape=component.signal_shape)
    #      for component in hps]))

class Layer:
  def __init__(self, hp:LayerHP):
    self.hp = hp
    self.patterns = PatchGridPatterns(patch_grid_shape=self.hp.output_shape, per_patch_pattern_grid_hp=self.hp.per_patch_pattern_grid_hp)
    self.output = SignalGrid(hp.output_hp)

  def forward(self, input:CompositeSignalGrid):
    assert input.grid_shape == (1,) # single signal input

    input_output_patch_grid = InputOutputPatchGrid(input=input, output=self.output, output_patch_shape=self.hp.output_neighborhood_shape, input_coverage_factor=1.0)
    # patches = LayerUtils.make_input_output_composite_patches(input=input, output=self.output, output_shape=self.hp.output_shape, output_neighborhood_shape=self.hp.output_neighborhood_shape, convolve_input=self.hp.convolve_input)

    pattern_similarity = PatternSimilarity(x=input_output_patch_grid.patches, y=self.patterns.patterns.composite_signal_grid)
    
    activation = pattern_similarity.sim
    activation = activation.max(dim=-1)[0].unsqueeze(dim=0)
    assert self.output.pixels.shape == activation.shape 

    self.output.pixels = soft_add(self.output.pixels, activation, tau=self.hp.output_tau)



In [ ]:
%aimport layer

In [ ]:

%%run_pytest[clean] -s
"""
Test Layer
"""

import pytest
from utils import *
from pattern import *
from layer import *

def test_layer_create():
  hp = LayerHP(pattern_grid_shape=(2,2), output_shape=(5,5), output_neighborhood_shape=(3,3), input_signal_shapes=[(10,15),(20,10)])
  layer = Layer(hp=hp)
  # pretty_print("Layer", layer)
  assert(True)

def test_layer_forward():
  input_signal_shapes = [(10,10), (20,20)]
  input_signals = [torch.ones(shape) for shape in input_signal_shapes]
  input = CompositeSignalGrid.from_tensors(input_signals)
  hp = LayerHP(pattern_grid_shape=(2,3), output_shape=(5,5), output_neighborhood_shape=(3,3), input_signal_shapes=input_signal_shapes)
  layer = Layer(hp=hp)
  #pretty_print("Layer", layer)

  for _ in range(10):
    layer.forward(input)

  assert torch.allclose(layer.output.pixels, torch.ones_like(layer.output.pixels), atol=0.01)

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.datasets import load_boston
from utils import *
from layer import *

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def normalize(df):
  df1 = (df - df.mean())/df.std()
  return df1

def scale(df):
  min = df.min()
  max = df.max()

  df1 = (df - min) / (max - min)
  return df1

dataset = load_boston()
dataset = pd.DataFrame(dataset.data, columns=dataset.feature_names)
dataset = pd.DataFrame(np.c_[scale(normalize(dataset['LSTAT'])), scale(normalize(dataset['RM']))], columns = ['LSTAT','RM'])
dataset = torch.tensor(dataset.to_numpy()).float().to(device)
dataset

input_signal_shapes = [(1,)]
hp = LayerHP(pattern_grid_shape=(10,10), output_shape=(1,), output_neighborhood_shape=(1,), input_signal_shapes=input_signal_shapes, convolve_input=False)
layer = Layer(hp=hp)

plot_patterns(patterns=None, pattern_lr=None, dataset=dataset, voronoi=False, annotate=False)


In [ ]:
import numpy as np

def make_ndarray(shape, fn):
  a = np.empty(shape, dtype=object)
  with np.nditer(a, flags=['refs_ok', 'multi_index'], op_flags=['readwrite']) as it:
    for x in it:
      a[it.multi_index] = fn(it.multi_index)

  return a

make_ndarray((2,3), lambda multi_index:multi_index)

array([[(0, 0), (0, 1), (0, 2)],
       [(1, 0), (1, 1), (1, 2)]], dtype=object)

We have patch grid of composite signal. Each composite signal gets its own grid of patterns.